# Improving the accuracy of BQSKit compiled circuits with error mitigation

In this tutorial we describe how to use error mitigation capabilities from [Mitiq](https://mitiq.readthedocs.io/en/stable/) together with the compilation capabilities of [BQSKit](https://bqskit.lbl.gov/), a compiler for quantum circuits. BQSKit stands for Berkeley Quantum Synthesis Toolkit and it allows one "to compile quantum programs to efficient physical circuits for any QPU".

To get started, ensure you have the requisite python packages by running the following install commands.
- `pip install mitiq`
- `pip install bqskit==0.3.0` (here we use an older version of `bqskit` to satisfy version requirements between the two packages)

The main goal of this tutorial is to understand how to use `bqskit` together with `mitiq`.
To do this, we will
1. generate a random circuit,
2. compile it with `bqskit`,
3. use error mitigation on the compiled circuit, and
4. compare the results obtained with and without error mitigation.

After demonstrating the use of the two packages, we can then try and understand how circuit compilation with BQSKit in general interacts with error mitigation by running the process many times and comparing results.

---

To begin we import many of the required modules and functions.

In [1]:
import bqskit
import mitiq

import cirq
import numpy as np
from cirq.contrib.qasm_import import circuit_from_qasm

## Random circuit generation

We use `cirq`'s [`random_circuit`](https://quantumai.google/reference/python/cirq/testing/random_circuit) function to generare a random circuit with specified qubit number, depth and density (which refers to the probability of an operation occuring at a given moment).
Here we also use a random seed for reproducibility.

In [2]:
num_qubits = 3
depth = 10
density = 1
RANDOM_SEED = 479

random_circuit = cirq.testing.random_circuit(
    num_qubits, depth, density, random_state=RANDOM_SEED
)

In [3]:
print(random_circuit)

                                          ┌──┐
0: ───X───Z───Y───X───────────────iSwap─────@────H───Y───
                  │               │         │
1: ───S───X───────@───Y───iSwap───┼────────S┼────X───X───
          │               │       │         │
2: ───────@───S───Y───Y───iSwap───iSwap─────@────────────
                                          └──┘


Since `bqskit` uses a custom intermediate representation for circuits, we must write the circuit out to QASM for consumption by `bqskit`.
Here we must remove the comment `// Generated from Cirq v1.0.0` at the beginning of the file to ensure `bqskit` can properly consume the operations described therein.

In [4]:
qasm_code = cirq.qasm(random_circuit)
with open("tmp.qasm", "w") as f:
    f.write(qasm_code[31:])  # remove google prefix

We now have a file `tmp.qasm` that contains a random circuit, ready for compilation.

## Compilation

The random circuit can then be read in by `bqskit`'s [`from_file`](https://bqskit.readthedocs.io/en/latest/source/autogen/bqskit.ir.Circuit.from_file.html) function in preparation for compilation.
The workflow is then to create a [`CompilationTask`](https://bqskit.readthedocs.io/en/latest/source/autogen/bqskit.compiler.CompilationTask.html) object which completely specifies the mathematical problem of the circuit compilation problem.
By default `bqskit` attempts to reduce the circuit depth as its primary optimization, and compiles the circuit into the following gateset: $\{\mathsf{U3}, \mathsf{CNOT}, \sqrt{X}, R_Z\}$.

In [5]:
bqs_circuit = bqskit.ir.Circuit.from_file("tmp.qasm")
task = bqskit.compiler.CompilationTask.optimize(bqs_circuit)

with bqskit.compiler.Compiler() as compiler:
    compiled_bqs_circuit = compiler.compile(task)

For further processing, we convert the compiled circuit back into a `cirq.Circuit` object.
This can be done easily by writing the circuit to QASM, which `cirq` can then read in.

In [6]:
compiled_bqs_circuit.save("out.qasm")

with open("out.qasm") as f:
    qasm_out = f.read()

compiled_circuit = circuit_from_qasm(qasm_out)

At this point we have two `cirq` circuits: `random_circuit` and `compiled_circuit`.
Both represent the same (or very close to the same) unitary operation, but with different gatesets.

## Error Mitigation

Now that we have a compiled circuit, we are ready to use `mitiq`'s error mitigation capabilities.
In this tutorial we will use one of the simplest, and easiest to use methods: [Zero Noise Extrapolation](https://mitiq.readthedocs.io/en/stable/guide/zne-1-intro.html) (ZNE), but there are multiple other techniques described in our [user guide](https://mitiq.readthedocs.io/en/stable/guide/guide.html) which could be used as well.
In this tutorial we assume a simple error model of depolarizing noise on two-qubit gates.

To use this method, we need to define a function (in `mitiq` this is often referred to as an executor) which takes as input a circuit, and returns some sort of expectation value, or probability.
Here we will define a function `execute` which adds a tunable noise parameter, which controls the strength of the simulated noise.
Then, a density matrix simulation is run, and we measure the probability of observing the system in the ground state(s).

In [7]:
def execute(circuit, noise_level=0.05):
    noisy_circuit = cirq.Circuit()
    for op in circuit.all_operations():
        noisy_circuit.append(op)
        if len(op.qubits) == 2:
            noisy_circuit.append(
                cirq.depolarize(p=noise_level, n_qubits=2)(*op.qubits)
            )

    rho = (
        cirq.DensityMatrixSimulator()
        .simulate(noisy_circuit)
        .final_density_matrix
    )
    return rho[0, 0].real

Since we'd like to see how compilation effects error mitigation, we first simulate the ideal and noisy values using the simulator defined above.

In [8]:
uncompiled_ideal_value = execute(random_circuit, noise_level=0.0)
uncompiled_noisy_value = execute(random_circuit)

compiled_ideal_value = execute(compiled_circuit, noise_level=0.0)
compiled_noisy_value = execute(compiled_circuit)

With these values taken, we are now ready to use ZNE --- on both the random, and compiled circuit --- to obtain mitigated expectation values.

In [9]:
from mitiq import zne

uncompiled_mitigated_result = zne.execute_with_zne(random_circuit, execute)
compiled_mitigated_result = zne.execute_with_zne(compiled_circuit, execute)

Thus we have four variables which we can compare against ideal values to see how performance varies for this circuit across compilation and mitigation.

|                                | compiled | mitigated |
| ------------------------------ | -------- | --------- |
| `uncompiled_noisy_value`       | ❌       | ❌         |
| `uncompiled_mitigated_result`  | ❌       | ✅         |
| `compiled_noisy_value`         | ✅       | ❌         |
| `compiled_mitigated_result`    | ✅       | ✅         |


## Comparison

These data are then summarized in the following table printed below.

In [10]:
header = "{:<11} {:<15} {:<10}"
entry = "{:<11}  {:<15.2f} {:<10.2f}"
int_entry = "{:<11}  {:<15} {:<10}"
print(header.format("", "uncompiled", "compiled"))
print(entry.format("ideal", uncompiled_ideal_value, compiled_ideal_value))
print(entry.format("noisy", uncompiled_noisy_value, compiled_noisy_value))
print(
    entry.format(
        "mitigated", uncompiled_mitigated_result, compiled_mitigated_result
    )
)
print(
    entry.format(
        "error",
        abs(uncompiled_ideal_value - uncompiled_mitigated_result),
        abs(compiled_ideal_value - compiled_mitigated_result),
    )
)
print(
    int_entry.format(
        "depth",
        len(random_circuit),
        len(compiled_circuit),
    )
)

            uncompiled      compiled  
ideal        0.50            0.50      
noisy        0.43            0.42      
mitigated    0.33            0.53      
error        0.17            0.03      
depth        10              13        


Hence for this particular random circuit we see that using both compilation _and_ error mitigation combine for the most accurate result.
Note that despite using BQSKit to compile the circuit, the depth has actually increased. 
This can occasionally happen when the random circuit contains gates that are harder to compile into BQSKit's default gateset.

## More random circuits

We can now repeat the above procedure with many random circuits to get a better understanding of how these two technologies interact in a more general setting.
To do this we execute the above code many times, each iteration using a new random circuit on 4 qubits with depth 40.
Because compiling many large circuits is computationally expensive, we leave the code our from this notebook, but it can be accessed in our [research repository](https://github.com/unitaryfund/research/blob/main/ieee-quantum-week/compilation-with-error-mitigation-tutorial/bqskit.ipynb).

Once the errors are computed for each circuit we can collect the results in a histogram to get an idea of how compilation and mitigation affects accuracy more generally.

<img src="../img/bqskit.png" alt="Histograms of circuit accuracy with and without compilation, and error mitigation." width="600"/>

These results show that using error mitigation improves the accuracy of both uncompiled, and compiled circuits.
The [tutorial](https://github.com/unitaryfund/research/blob/main/ieee-quantum-week/compilation-with-error-mitigation-tutorial/bqskit.ipynb) in the research repository shows further that error mitigation both reduces the mean, and standard deviation of these distributions.

In this tutorial we've seen how one can use error mitigation in conjunction with circuit compilation.
For more information check out the [`bqskit`](https://bqskit.readthedocs.io/en/latest/) and [`mitiq`](https://mitiq.readthedocs.io/en/stable/) documentation.

### References

BQSKit documentation: <https://bqskit.readthedocs.io/>

BQSKit whitepaper: <https://dl.acm.org/doi/abs/10.1145/3503222.3507739/>

Mitiq documentation: <https://mitiq.readthedocs.io/>

Mitiq whitepaper: <https://quantum-journal.org/papers/q-2022-08-11-774/>